# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
%%time
data = get_date_list(data)
data = read_data(data)

data['df'].head(2)

Date List Created
Record num : 5000
Wall time: 14.6 s


,DateTime,Bid,Ask,Volume,i,tick
0,20220718 00:00:00.247,1.00972,1.00978,3,0,0.0
1,20220718 00:00:00.349,1.00972,1.00977,2,0,0.0


In [3]:
data['candle_size'] = '5T'

In [4]:
data['df']['tick']      = (data["df"]['Ask'] + data["df"]['Bid'])/2
data['df']['DateTime_frmt']   = [dt.datetime.strptime(x.split(".")[0],"%Y%m%d %H:%M:%S") for x in data["df"]['DateTime']]
data['df'] = data['df'][['DateTime_frmt', 'tick']]
data['df'] = data['df'].set_index('DateTime_frmt')
data['df'] = data['df'].resample(data['candle_size']).ohlc(_method='ohlc')

col_names = [col[1] for col in data['df'].columns.values]
col_names.insert(0,'DateTime_frmt')
data['df'] = pd.DataFrame(data['df'].to_records())
data['df'].columns = col_names

data['df'].head(2)

tick                            
                        open     high       low    close
DateTime_frmt                                           
2022-07-18 00:00:00  1.00975  1.00978  1.009550  1.00969
2022-07-18 00:05:00  1.00970  1.00988  1.009425  1.00947

def get_ohlc(data):
    
    data['df']['o'] = np.nan
    data['df']['h'] = np.nan
    data['df']['l'] = np.nan
    data['df']['c'] = np.nan    
    
    data['rolled_index'] = data['df'].resample(data['candle_size']).tick.first().index

    for i in np.arange(1,len(data['rolled_index'])):
                            
        timestamp_val = data['df'].loc[data['rolled_index'][i-1]:data['rolled_index'][i]]['tick'].index[-1]
        
        tick_val      = data['df'].loc[data['rolled_index'][i-1]:data['rolled_index'][i]]['tick'].values[0]
        data['df']['o'].loc[timestamp_val] = tick_val
        
        tick_val      = data['df'].loc[data['rolled_index'][i-1]:data['rolled_index'][i]]['tick'].values.max()
        data['df']['h'].loc[timestamp_val] = tick_val
        
        tick_val      = data['df'].loc[data['rolled_index'][i-1]:data['rolled_index'][i]]['tick'].values.min()
        data['df']['l'].loc[timestamp_val] = tick_val

        tick_val      = data['df'].loc[data['rolled_index'][i-1]:data['rolled_index'][i]]['tick'].values[0]
        data['df']['c'].loc[timestamp_val] = tick_val
        
        data['df'] = data['df'].reset_index()
        data['df'] = data['df'].drop_duplicates(subset = ['DateTime_frmt'], keep = 'last')
    
#     data['df'] = data['df'][['sno', 'i', 'DateTime','Ask', 'Bid', 'tick', 'sema', 'lema', 'slema', 'o', 'h', 'l', 'c', ]].round(6)

    return(data)